In [2]:
#to create dataset
lll = []
import cv2
import mediapipe as mp
k=0
for i in range(42):#repreasent the number of video in the folder
    class Point:
        X = 0
        Y = 0

        def __init__(self, X=0, Y=0):
            self.X = X
            self.Y = Y

        def Add(self, Point):
            self.X += Point.X
            self.Y += Point.Y

        def SubtractPawer2(Point1, Point2):
            x = Point()
            x.X = Point1.X - Point2.X
            x.Y = Point1.Y - Point2.Y
            x.X *= x.X
            x.Y *= x.Y
            return x


    class Movment:
        PointsList = []
        Avarage = None
        Variance = None

        def init(self):
            self.Variance = Point()
            self.Avarage = Point()

        def AddPoint(self, dot, CenterPoint, index):
            PointX, PointY = dot
            CenterX, CenterY = CenterPoint
            dot = Point(abs(CenterX-PointX), abs(CenterY-PointY))
            self.PointsList.append(dot)
            count = len(self.PointsList)
            if self.PointsList[count-1].X == CenterX:
                self.PointsList[count-1].X = self.PointsList[count-2].X
            if self.PointsList[count-1].Y == CenterY:
                self.PointsList[count-1].Y = self.PointsList[count-2].Y
            if count > 2:
                TotalX = self.PointsList[count-1].X + self.PointsList[count-3].X
                TotalX -= 2*self.PointsList[count-2].X
                TotalY = self.PointsList[count-1].Y + self.PointsList[count-3].Y
                TotalY -= 2*self.PointsList[count-2].Y
                Total = TotalX + TotalY
                return Total
            return 0

    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    k+=1
    v=''
    #file_path=f"C:/Users/Seif Elmoushy/Desktop/7 techniqes/good/{k}.mp4"
    file_path=f"C:/Users/Seif Elmoushy/Downloads/Dataset/bad/{k}.mp4"
    MovmentList = [Movment() for _ in range(33)]
    # Initialize video capture
    cap = cv2.VideoCapture(file_path)
    check_list = []
    check_list.append(0)
    check_list.append(0)
    flag = 0
    Means = []
    # Initialize Mediapipe pose detection
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter('output.mp4', fourcc, 30.0,
                              (int(cap.get(3)), int(cap.get(4))))

        landmark_coords = []
        while cap.isOpened():
            ret, image = cap.read()

            if not ret:
                break

            image = cv2.flip(image, 1)

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            results = pose.process(image)

            if results.pose_landmarks is not None:

                image_height, image_width, _ = image.shape
                center_x = int(sum(
                    [landmark.x for landmark in results.pose_landmarks.landmark]) / 33 * image.shape[1])

                center_y = int(sum(
                    [landmark.y for landmark in results.pose_landmarks.landmark]) / 33 * image.shape[0])

                cv2.circle(image, (center_x, center_y), 5, (255, 0, 0), -1)  # Red
                Diff = 0
                for index, dot in enumerate(results.pose_landmarks.landmark):
                    X = int(dot.x*image_width)
                    Y = int(dot.y*image_height)
                    Diff += MovmentList[index].AddPoint(
                        (X, Y), (center_x, center_y), index)
                if abs(Diff) > 3:
                    cv2.putText(image, "True", (50, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    flag = 1
                else:
                    cv2.putText(image, "False", (50, 50),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                    flag = 0
                out.write(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            check_list.append(flag)
            if results.pose_landmarks:
                    landmark_coords.append(results.pose_landmarks.landmark)
            if check_list[-2] == 0 and check_list[-1] == 0:
                landmark_coords.clear()
            if check_list[-2] == 1 and check_list[-1] == 0:
                if len(landmark_coords)<1:
                    landmark_coords.clear()
                else:
                    lll.append(landmark_coords.copy())
                    one_d_list = []
                    Means = [{'X': 0, 'Y': 0, 'Z': 0, "Visability": 0}
                             for _ in range(33)]
                    for sublist in landmark_coords:
                        for i, dot in enumerate(sublist):
                            Means[i]['X'] += dot.x
                            Means[i]['Y'] += dot.y
                            Means[i]['Z'] += dot.z
                            Means[i]['Visability'] += dot.visibility
                    for i in range(len(Means)):
                        Means[i]['X'] /= len(landmark_coords)
                        Means[i]['Y'] /= len(landmark_coords)
                        Means[i]['Z'] /= len(landmark_coords)
                        Means[i]['Visability'] /= len(landmark_coords)
                    lll.append(Means)
                    landmark_coords.clear()

            cv2.imshow('Mediapipe Object Centering', image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    out.release()
    cap.release()
    cv2.destroyAllWindows()

In [26]:
num_coords=33
landmark=[]
for val in range(1, num_coords+1):
    landmark += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
import csv
with open("C:/Users/Seif Elmoushy/Downloads/Dataset/frist katapoints.csv", mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmark)
    for i in range(len(lll)):
        List = []
        for j in range(len(lll[i])):
            List.extend([lll[i][j]['X'], lll[i][j]['Y'],lll[i][j]['Z'],lll[i][j]['Visability']])
        csv_writer.writerow(List)
        

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("C:/Users/Seif Elmoushy/Downloads/Dataset/frist katapoints.csv")

X = df.drop(['class'], axis=1)
y = df['class']

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=42)

svm = SVC()

param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']
}

grid_search = GridSearchCV(svm, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Best parameters: ", grid_search.best_params_)
svm = grid_search.best_estimator_

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

AvgAcc = 0
AvgRecall = 0
AvgPrecision = 0
AvgF1 = 0
fold = 0

for train_index, test_index in kfold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    AvgAcc += accuracy
    
    recall = recall_score(y_test, y_pred, average='macro')
    AvgRecall += recall
    
    precision = precision_score(y_test, y_pred, average='macro')
    AvgPrecision += precision
    
    f1 = f1_score(y_test, y_pred, average='macro')
    AvgF1 += f1 
    
    cnf_matrix = confusion_matrix(y_test, y_pred)
    print("Fold", str(fold))
    print("\tAccuracy: "+str(accuracy)+", recall: "+str(recall)+", precision: "+str(precision)+", f1: "+str(f1))
    print("\tConfusion matrix:\n", cnf_matrix)
    fold += 1

AvgAcc /= fold
AvgRecall /= fold 
AvgPrecision /= fold
AvgF1 /= fold
print("Average")
print("\tAccuracy: "+str(AvgAcc)+", recall: "+str(AvgRecall) +
      ", precision: "+str(AvgPrecision)+", f1: "+str(AvgF1))


Best parameters:  {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
Fold 0
	Accuracy: 0.9449715370018975, recall: 0.8617726000799497, precision: 0.9110966904607398, f1: 0.8840484048404841
	Confusion matrix:
 [[440   9]
 [ 20  58]]
Fold 1
	Accuracy: 0.9335863377609108, recall: 0.805589970501475, precision: 0.9055868765171091, f1: 0.8454221663523989
	Confusion matrix:
 [[445   7]
 [ 28  47]]
Fold 2
	Accuracy: 0.9411764705882353, recall: 0.7997680228408279, precision: 0.8872521899492853, f1: 0.8360479702945456
	Confusion matrix:
 [[459   8]
 [ 23  37]]
Fold 3
	Accuracy: 0.9543726235741445, recall: 0.8900221729490023, precision: 0.9183712301398534, f1: 0.9034508076358296
	Confusion matrix:
 [[442   9]
 [ 15  60]]
Fold 4
	Accuracy: 0.9220532319391636, recall: 0.8028701662993387, precision: 0.9145839761390517, f1: 0.8448879778473046
	Confusion matrix:
 [[428   6]
 [ 35  57]]
Fold 5
	Accuracy: 0.9429657794676806, recall: 0.8699254768220286, precision: 0.9235530616376941, f1: 0.8937459599224304
	Con